#  Northwind BI Dashboard



In [9]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import os
import itertools



#root_wassim


try:
    
    CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
  
    CURRENT_DIR = os.getcwd()

if 'notebooks' in CURRENT_DIR:
    BASE_DIR = os.path.dirname(CURRENT_DIR)
else:
    BASE_DIR = CURRENT_DIR

WAREHOUSE_DIR = os.path.join(BASE_DIR, 'data', 'warehouse')
print(f' Loading data from {WAREHOUSE_DIR}')


try:
    fact_sales = pd.read_parquet(os.path.join(WAREHOUSE_DIR, 'FactSales.parquet'))
    dim_date = pd.read_parquet(os.path.join(WAREHOUSE_DIR, 'DimDate.parquet'))
    dim_emp = pd.read_parquet(os.path.join(WAREHOUSE_DIR, 'DimEmployee.parquet'))
    dim_client = pd.read_parquet(os.path.join(WAREHOUSE_DIR, 'DimClient.parquet'))

   
    if 'sk_date' not in dim_date.columns and 'date_key' in dim_date.columns:
        dim_date = dim_date.rename(columns={'date_key': 'sk_date'})

    
    dim_emp = dim_emp.rename(columns={
        'city': 'emp_city', 
        'country': 'emp_country', 
        'region': 'emp_region', 
        'title': 'emp_title'
    })
    
   
    dim_client = dim_client.rename(columns={'region': 'client_region'})


    df = pd.merge(fact_sales, dim_date, on='sk_date', how='left')
    df = pd.merge(df, dim_emp, on='sk_employee', how='left')
    df = pd.merge(df, dim_client, on='sk_client', how='left')

   


    if 'full_name' in df.columns:
        df = df.rename(columns={'full_name': 'Employee_name'})
    elif 'first_name' in df.columns:
       
        df['Employee_name'] = df['first_name'].astype(str) + ' ' + df['last_name'].astype(str)

   
    df['Employee_name'] = df['Employee_name'].fillna('Unknown Employee')
    df['company_name'] = df['company_name'].fillna('Unknown Client')

    
    df['date_obj'] = pd.to_datetime(df['full_date'])
    df['year_month'] = df['date_obj'].dt.strftime('%Y-%m')

    
    min_year = int(df['year'].min())
    max_year = int(df['year'].max())
    available_years = list(range(min_year, max_year + 1))
    year_options = ['All Years'] + [str(y) for y in available_years]

    print(f' Dashboard Ready. Total Records {len(df)}')
    print(f' Years Available {year_options}')



except Exception as e:
   
    print(f' Error Loading Data: {e}')
  
    import traceback
    traceback.print_exc()

 Loading data from c:\Users\IT7\Desktop\Bi-Northwind-project\Bi-Northwind-project\data\warehouse
 Dashboard Ready. Total Records 2223
 Years Available ['All Years', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006']


In [10]:

kpi_dropdown = widgets.Dropdown(options=year_options, value='All Years', description='Date:')
kpi_output = widgets.Output()

def update_kpi(change):
    selected_year = change['new']
    with kpi_output:
        clear_output(wait=True)
        if selected_year == 'All Years': data = df.copy()
        else: data = df[df['year'] == int(selected_year)]
            
        if data.empty:
            display(Markdown("### No data for this period"))
            return

     
        rev = data['total_amount'].sum()
        
    
        uniq_orders = data.drop_duplicates(subset=['bk_order_id'])
        total_orders = len(uniq_orders)
        
        shipped = len(uniq_orders[uniq_orders['delivery_status'] == 'Livrée'])
        pending = len(uniq_orders[uniq_orders['delivery_status'] == 'Non Livrée'])
        
        shipped_pct = (shipped / total_orders * 100) if total_orders > 0 else 0
        pending_pct = (pending / total_orders * 100) if total_orders > 0 else 0

       
        fig = make_subplots(
            rows=1, cols=4, 
            specs=[[{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}]]
        )

   
        fig.add_trace(go.Indicator(
            mode="number", 
            value=rev, 
            title={"text": "Total Revenue", "font": {"size": 20, "color": "gray"}},
            number={'prefix': "$", "valueformat": ",.0f", "font": {"size": 50, "color": "#636EFA"}}
        ), row=1, col=1)

    
        fig.add_trace(go.Indicator(
            mode="number", 
            value=total_orders, 
            title={"text": "Total Orders", "font": {"size": 20, "color": "gray"}},
            number={"valueformat": ",", "font": {"size": 50, "color": "#444"}}
        ), row=1, col=2)

        

        fig.add_trace(go.Indicator(
            mode="number+gauge", 
            value=shipped_pct, 
            title={"text": f"Delivered ({shipped})", "font": {"size": 14}},
            number={'suffix': "%", "font": {"size": 30, "color": "#00CC96"}},
            gauge={
                'axis': {'range': [0, 100]},
                'bar': {'color': "#00CC96"},
                'bgcolor': "white",
                'borderwidth': 2,
                'bordercolor': "#eee"
            }
        ), row=1, col=3)

   
        fig.add_trace(go.Indicator(
            mode="number+gauge", 
            value=pending_pct, 
            title={"text": f"Pending ({pending})", "font": {"size": 14}},
            number={'suffix': "%", "font": {"size": 30, "color": "#EF553B"}},
            gauge={
                'axis': {'range': [0, 100]},
                'bar': {'color': "#EF553B"},
                'bgcolor': "white",
                'borderwidth': 2,
                'bordercolor': "#eee"
            }
        ), row=1, col=4)

        

        
        fig.update_layout(height=250, margin=dict(l=20, r=20, t=30, b=20))
        fig.show()

kpi_dropdown.observe(update_kpi, names='value')
display(Markdown("### Executive Summary"), kpi_dropdown, kpi_output)
update_kpi({'new': 'All Years'})

### Executive Summary

Dropdown(description='Date:', options=('All Years', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2…

Output()

In [11]:

emp_dropdown = widgets.Dropdown(options=year_options, value='All Years', description='Date:')
emp_output = widgets.Output()

def update_employee_delivery(change):
    selected_year = change['new']
    with emp_output:
        clear_output(wait=True)
        
        if selected_year == 'All Years': data = df.copy()
        else: data = df[df['year'] == int(selected_year)]
            
   
        emp_stats = data.drop_duplicates(subset=['bk_order_id']).groupby(['Employee_name', 'delivery_status']).size().reset_index(name='count')
        emp_stats = emp_stats.sort_values(by='count')

        fig = px.bar(
            emp_stats, x='count', y='Employee_name', color='delivery_status',
            title=f"Employee Delivery Performance ({selected_year})",
            barmode='stack',
            orientation='h',
            color_discrete_map={'Livrée': "#2B8BD5", 'Non Livrée': "#EA492C"}
        )
        fig.update_layout(height=500)
        fig.show()

emp_dropdown.observe(update_employee_delivery, names='value')
display(Markdown("###  Employee Logistics Performance"), emp_dropdown, emp_output)
update_employee_delivery({'new': 'All Years'})

###  Employee Logistics Performance

Dropdown(description='Date:', options=('All Years', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2…

Output()

In [12]:

del_date_dropdown = widgets.Dropdown(options=year_options, value='All Years', description='Date:')
del_date_output = widgets.Output()



def update_del_date(change):
    selected_year = change['new']
    with del_date_output:
        clear_output(wait=True)
        
        
        if selected_year == 'All Years':
            start_date = df['date_obj'].min()
            end_date = df['date_obj'].max()
            freq = 'MS'
            format_str = '%Y-%m'
        else:
            start_date = pd.Timestamp(f"{selected_year}-01-01")
            end_date = pd.Timestamp(f"{selected_year}-12-31")
            freq = 'MS'
            format_str = '%B'

      
        if selected_year == 'All Years':
            data = df.copy()
            data['time_key'] = data['year_month']
        else:
            data = df[df['year'] == int(selected_year)].copy()
            
            data['month_num'] = data['date_obj'].dt.month 
            data['time_key'] = data['date_obj'].dt.strftime('%B')

      
        actual_counts = data.drop_duplicates(subset=['bk_order_id']).groupby(['time_key', 'delivery_status']).size().reset_index(name='count')

       
        full_dates = pd.date_range(start=start_date, end=end_date, freq=freq).strftime(format_str)
        statuses = ['Livrée', 'Non Livrée']
        
       
        full_grid = pd.MultiIndex.from_product([full_dates, statuses], names=['time_key', 'delivery_status']).to_frame(index=False)
        
       
        merged = pd.merge(full_grid, actual_counts, on=['time_key', 'delivery_status'], how='left')
        merged['count'] = merged['count'].fillna(0)

       
        fig = px.area(
            merged, 
            x='time_key', 
            y='count', 
            color='delivery_status',
            title=f"Delivery Trends ({selected_year})",
            color_discrete_map={'Livrée': "#2B8BD5", 'Non Livrée': "#EA492C"},
            line_shape='spline', 
            markers=True         
        )
        
        fig.update_layout(
            hovermode="x unified", 
            xaxis_title="Timeline",
            yaxis_title="Total Orders"
        )
        fig.show()

del_date_dropdown.observe(update_del_date, names='value')
display(Markdown("### Delivery Trend Analysis"), del_date_dropdown, del_date_output)
update_del_date({'new': 'All Years'})

### Delivery Trend Analysis

Dropdown(description='Date:', options=('All Years', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2…

Output()

In [13]:

client_dropdown = widgets.Dropdown(options=year_options, value='All Years', description='Date:')
client_output = widgets.Output()

def update_client_delivery(change):
    selected_year = change['new']
    with client_output:
        clear_output(wait=True)
        
      
        if selected_year == 'All Years': 
            data = df.copy()
        else: 
            data = df[df['year'] == int(selected_year)]
            
        if data.empty:
            print(f" No sales found for {selected_year}.")
            return

        client_stats = data.drop_duplicates(subset=['bk_order_id']).groupby(['company_name', 'delivery_status']).size().reset_index(name='count')
        
        if client_stats.empty:
            print(f" Data found, but no client stats generated.")
            return

     

        fig = px.treemap(
            client_stats, 
            path=['company_name', 'delivery_status'], 
            values='count',
            color='delivery_status',
            title=f"Delivery Status Distribution ({selected_year})",
            color_discrete_map={'Livrée': "#2B8BD5", 'Non Livrée': "#EA492C"},
            hover_data=['count']
        )
        
        

        fig.update_layout(height=600, margin=dict(l=10, r=10, t=40, b=10)) 
        fig.show()

client_dropdown.observe(update_client_delivery, names='value')
display(Markdown(" Client Delivery Analysis (Treemap)"), client_dropdown, client_output)
update_client_delivery({'new': 'All Years'})

 Client Delivery Analysis (Treemap)

Dropdown(description='Date:', options=('All Years', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2…

Output()

In [14]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import itertools

if 'Employee_name' not in df.columns:
    df['Employee_name'] = df['first_name'].astype(str) + " " + df['last_name'].astype(str)


df['company_name'] = df['company_name'].fillna('Unknown Client')
df['Employee_name'] = df['Employee_name'].fillna('Unknown Employee')
df['year_month'] = df['date_obj'].dt.strftime('%Y-%m')

 
olap_dropdown = widgets.Dropdown(options=year_options, value='All Years', description='Date:')
olap_output = widgets.Output()

def update_olap(change):
    selected_year = change['new']
    with olap_output:
        clear_output(wait=True)
        
      
        if selected_year == 'All Years':
            data = df.copy()
            time_col = 'year'
          
            min_y, max_y = int(data['year'].min()), int(data['year'].max())
            full_timeline = [str(y) for y in range(min_y, max_y + 1)]
            xlabel = 'Year'
        else:
            data = df[df['year'] == int(selected_year)].copy()
            time_col = 'year_month'
      
            full_timeline = [f"{selected_year}-{m:02d}" for m in range(1, 13)]
            xlabel = 'Month'

        if data.empty and selected_year == 'All Years':
            display(Markdown("###  No data available."))
            return

      
        
        if not data.empty:
        
            top_clients = data.groupby('company_name')['total_amount'].sum().nlargest(10).index.tolist()
            employees = data['Employee_name'].unique().tolist()
        else:
            top_clients = []
            employees = []
        
   
        grid = pd.DataFrame(list(itertools.product(full_timeline, top_clients, employees)), 
                            columns=[time_col, 'company_name', 'Employee_name'])
        
  
        data[time_col] = data[time_col].astype(str).str.replace(r'\.0', '', regex=True)


        actuals = data.groupby([time_col, 'company_name', 'Employee_name'])['total_amount'].sum().reset_index()
        
      
        df_dense = pd.merge(grid, actuals, on=[time_col, 'company_name', 'Employee_name'], how='left')
        df_dense['total_amount'] = df_dense['total_amount'].fillna(0)

      
        df_zeros = df_dense[df_dense['total_amount'] == 0]
        df_sales = df_dense[df_dense['total_amount'] > 0]  

        fig = px.scatter_3d(
            df_sales, 
            x=time_col, 
            y='company_name', 
            z='Employee_name',
            size='total_amount', 
            color='total_amount', 
            opacity=0.9,
            color_continuous_scale='Portland',
            title=f"3D OLAP:{xlabel} x Client x Employee",
            labels={'total_amount': 'Revenue', time_col: xlabel, 'company_name': 'Client', 'Employee_name': 'Employee'}
        )

        fig.add_trace(go.Scatter3d(
            x=df_zeros[time_col],
            y=df_zeros['company_name'],
            z=df_zeros['Employee_name'],
            mode='markers',
            marker=dict(size=4, color='lightgrey', opacity=0.3, symbol='circle'),
            name=' No Orders (Gap)'
        ))
        
       
        fig.update_layout(
            height=700, 
            margin=dict(l=0, r=0, b=0, t=40),
            showlegend=True,
            legend=dict(x=0, y=1, bgcolor="rgba(255,255,255,0.8)", borderwidth=1),
            scene=dict(
                xaxis=dict(
                    title=xlabel, 
                    type='category', 
                    categoryorder='array', 
                    categoryarray=full_timeline,
                    tickmode='linear', 
                    dtick=1
                ),
                yaxis=dict(title='Client'),
                zaxis=dict(title='Employee')
            )
        )
        fig.show()

olap_dropdown.observe(update_olap, names='value')
display(Markdown("###  3D OLAP Analysis ( Time, Client, Employee)"), olap_dropdown, olap_output)
update_olap({'new': 'All Years'})

###  3D OLAP Analysis ( Time, Client, Employee)

Dropdown(description='Date:', options=('All Years', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2…

Output()

In [15]:
 
import pandas as pd
import plotly.express as px
from IPython.display import display, Markdown
import os



try:
    CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    CURRENT_DIR = os.getcwd()

if 'notebooks' in CURRENT_DIR: 
    BASE_DIR = os.path.dirname(CURRENT_DIR)
else: 
    BASE_DIR = CURRENT_DIR

RAW_DIR = os.path.join(BASE_DIR, 'data', 'raw')

try:
 
    raw_emp_terr = pd.read_csv(os.path.join(RAW_DIR, 'sql_employeeterritories.csv'))
    raw_terr = pd.read_csv(os.path.join(RAW_DIR, 'sql_territories.csv'))
    
   
    raw_region = pd.read_csv(os.path.join(RAW_DIR, 'sql_region.csv'))

   
    raw_emp_terr.columns = [c.lower().strip() for c in raw_emp_terr.columns]
    raw_terr.columns = [c.lower().strip() for c in raw_terr.columns]
    raw_region.columns = [c.lower().strip() for c in raw_region.columns]

   
    emp_geo = pd.merge(dim_emp, raw_emp_terr, left_on='bk_employee_id', right_on='employeeid', how='inner')
    
   
    emp_geo = pd.merge(emp_geo, raw_terr, on='territoryid', how='inner')
    
    emp_geo = pd.merge(emp_geo, raw_region, on='regionid', how='inner')


    split_view = emp_geo[['Employee_name', 'emp_title', 'regiondescription', 'territorydescription']].copy()
    split_view.columns = ['Employee Name', 'Title', 'Region', 'Territory']
    
    split_view = split_view.sort_values(by=['Employee Name', 'Territory'])

    
    fig = px.sunburst(
        split_view, 
        path=['Employee Name', 'Territory'], 
        title="Territory Distribution by Employee",
        color='Employee Name', 
      
        color_discrete_sequence=px.colors.qualitative.Prism 
    )
    
    fig.update_layout(height=600, margin=dict(t=40, l=0, r=0, b=0))
    fig.show()

except Exception as e:
    print(f"Could not load territory details: {e}")
    print("Ensure you extracted 'sql_employeeterritories.csv' and 'sql_territories.csv'.")

In [16]:

trend_dropdown = widgets.Dropdown(options=year_options, value='All Years', description=' Date:')
trend_output = widgets.Output()

def update_trend(change):
    selected_year = change['new']
    with trend_output:
        clear_output(wait=True)
        
       
        if selected_year == 'All Years':
            data = df.copy()
            x_col = 'year' 
            x_label = 'Year'
            title_text = "Yearly Revenue Evolution"
           
            trend_df = data.groupby(x_col)['total_amount'].sum().reset_index()
            trend_df = trend_df.sort_values('year')
        else:
            data = df[df['year'] == int(selected_year)].copy()
            x_col = 'month_name'
            x_label = 'Month'
            title_text = f"Monthly Revenue Trend ({selected_year})"
            
           
            data['month_num'] = data['date_obj'].dt.month
            data['month_name'] = data['date_obj'].dt.strftime('%B')
            
            
            trend_df = data.groupby(['month_num', 'month_name'])['total_amount'].sum().reset_index()
            trend_df = trend_df.sort_values('month_num') 
            x_col = 'month_name'

        
        avg_rev = trend_df['total_amount'].mean()

       
        fig = go.Figure()

      
        fig.add_trace(go.Bar(
            x=trend_df[x_col],
            y=trend_df['total_amount'],
            name="Revenue",
            marker_color='#636EFA',
            opacity=0.6
        ))

      
        fig.add_trace(go.Scatter(
            x=trend_df[x_col],
            y=trend_df['total_amount'],
            name="Trend",
            mode='lines+markers',
            line=dict(color='#00CC96', width=3),
            marker=dict(size=8)
        ))

      
        fig.add_hline(y=avg_rev, line_dash="dot", line_color="gray", 
                      annotation_text=f"Avg: ${avg_rev:,.0f}", annotation_position="top left")

        
        fig.update_layout(
            title=title_text,
            xaxis_title=x_label,
            yaxis_title="Revenue ($)",
            hovermode="x unified",
            showlegend=False,
            height=500
        )
        
        fig.show()

trend_dropdown.observe(update_trend, names='value')
display(Markdown("### Revenue Evolution"), trend_dropdown, trend_output)
update_trend({'new': 'All Years'})

### Revenue Evolution

Dropdown(description=' Date:', options=('All Years', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '…

Output()